# API REQUESTS

In [ ]:
import requests
import pandas as pd
import os
from dotenv import load_dotenv, find_dotenv
import time

In [ ]:
load_dotenv() # moves all variables from .env file to notebook memory.

my_key = os.getenv("API_KEY")


## Hotels in Lisbon:

In [ ]:
# Hotels in Lisbon

url = f"https://api.content.tripadvisor.com/api/v1/location/search?key={my_key}&searchQuery=Lisbon&category=hotels&language=en"

headers = {"accept": "application/json"}

response = requests.get(url, headers=headers)

print(response.text)

In [ ]:
load_dotenv()
API_KEY = os.getenv("API_KEY")

SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"
DETAILS_URL = "https://api.content.tripadvisor.com/api/v1/location/{}/details"

headers = {"accept": "application/json"}

queries = [
    "Lisbon hotel",
    "Lisbon hotels",
    "Lisbon accommodation",
    "Lisbon boutique hotel",
    "Lisbon luxury hotel",
    "Lisbon guesthouse",
    "Lisbon bed and breakfast",
]

all_hotels = []


for q in queries:
    url = (
        f"{SEARCH_URL}?key={API_KEY}"
        f"&searchQuery={q}"
        f"&category=hotels"
        f"&language=en"
    )

    response = requests.get(url, headers=headers)
    data = response.json().get("data", [])
    all_hotels.extend(data)

# Convert to unique hotel list
df = pd.DataFrame(all_hotels).drop_duplicates(subset=["location_id"])
df.reset_index(drop=True, inplace=True)

print(f"\n Total unique hotels collected: {len(df)}")
print(" Fetching rating details for each hotel...\n")

# Fetch ratings:
hotel_rows = []

for i, hotel in df.iterrows():
    loc_id = hotel["location_id"]
    detail_url = DETAILS_URL.format(loc_id)

    params = {"key": API_KEY, "language": "en", "currency": "EUR"}

    detail_res = requests.get(detail_url, headers=headers, params=params)
    if detail_res.status_code != 200:
        continue

    details = detail_res.json()

    hotel_rows.append({
        "location_id": loc_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "num_reviews": details.get("num_reviews"),
        "ranking": details.get("ranking"),
        "price_level": details.get("price_level"),
        "street1": details.get("address_obj", {}).get("street1"),
        "city": details.get("address_obj", {}).get("city"),
        "country": details.get("address_obj", {}).get("country"),
        "postalcode": details.get("address_obj", {}).get("postalcode"),
        "address_string": details.get("address_obj", {}).get("address_string"),
    })


    time.sleep(0.2)  # avoid rate limits

# Final DataFrame
df_hotels = pd.DataFrame(hotel_rows)

# Save CSV
output_file = "lisbon_hotels.csv"
df_hotels.to_csv(output_file, index=False)


- I got 48 hotels extracted

## Restaurants in Lisbon:

In [ ]:
# Load API key
load_dotenv()
API_KEY = os.getenv("API_KEY")

SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"
DETAILS_URL = "https://api.content.tripadvisor.com/api/v1/location/{}/details"

headers = {"accept": "application/json"}

# Expand the search to collect the maximum number of restaurants
queries = [
    "Lisbon restaurant",
    "restaurants Lisbon",
    "Lisbon food",
    "Lisbon dining",
    "Lisbon cafe",
    "Lisbon brunch",
    "Lisbon lunch",
    "Lisbon dinner",
    "Lisbon Michelin",
    "Lisbon traditional food",
    "Lisbon cheap eats",
]

all_restaurants = []

# Searching Restaurants:

for q in queries:
    url = (
        f"{SEARCH_URL}?key={API_KEY}"
        f"&searchQuery={q}"
        f"&category=restaurants"
        f"&language=en"
    )

    response = requests.get(url, headers=headers)

    if response.status_code != 200:
        print(f"Error for '{q}' → {response.text}")
        continue

    data = response.json().get("data", [])
    all_restaurants.extend(data)

    print(f"'{q}' returned {len(data)} restaurants")


# Deduplicate by TripAdvisor Location ID
df = pd.DataFrame(all_restaurants).drop_duplicates(subset=["location_id"])
df.reset_index(drop=True, inplace=True)

# Details for each Restaurant: 

restaurant_rows = []

for i, item in df.iterrows():

    loc_id = item["location_id"]
    detail_url = DETAILS_URL.format(loc_id)

    params = {
        "key": API_KEY,
        "language": "en",
        "currency": "EUR"
    }

    detail_res = requests.get(detail_url, headers=headers, params=params)

    if detail_res.status_code != 200:
        print(f"Details error for {loc_id}")
        continue

    details = detail_res.json()

    # Extract details safely
    address = details.get("address_obj", {}) or {}

    restaurant_rows.append({
        "location_id": loc_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "num_reviews": details.get("num_reviews"),
        "ranking": details.get("ranking"),
        "price_level": details.get("price_level"),
        "cuisine": ", ".join([c["name"] for c in details.get("cuisine", [])]) if details.get("cuisine") else None,
        "street1": address.get("street1"),
        "city": address.get("city"),
        "country": address.get("country"),
        "postalcode": address.get("postalcode"),
        "address_string": address.get("address_string"),
    })

    time.sleep(0.2)  # Prevent rate limit issues


# Saving Dataframe:

df_restaurants = pd.DataFrame(restaurant_rows)

output_file = "lisbon_restaurants.csv"
df_restaurants.to_csv(output_file, index=False)

print(f"\nDONE! Saved {len(df_restaurants)} restaurants with ratings to {output_file}")


- I got 62 restaurants extracted.

## Actractions in Lisbon:

In [ ]:
# Load API key
load_dotenv()
API_KEY = os.getenv("API_KEY")

SEARCH_URL = "https://api.content.tripadvisor.com/api/v1/location/search"
DETAILS_URL = "https://api.content.tripadvisor.com/api/v1/location/{}/details"

headers = {"accept": "application/json"}

# Search queries to expand coverage of attractions:
queries = [
    "Lisbon attractions",
    "best things to do Lisbon",
    "Lisbon museum",
    "Lisbon landmark",
    "Lisbon sightseeing",
    "Lisbon tourist spots",
    "Lisbon monuments",
    "Lisbon historical sites",
    "Lisbon outdoors",
    "Lisbon parks",
    "Lisbon viewpoints",
]

all_attractions = []

# Searching for attractions: 

for q in queries:
    url = (
        f"{SEARCH_URL}?key={API_KEY}"
        f"&searchQuery={q}"
        f"&category=attractions"
        f"&language=en"
    )

    response = requests.get(url, headers=headers)
    data = response.json().get("data", [])
    all_attractions.extend(data)

    print(f"'{q}' returned {len(data)} attractions")


# Deduplicate by TripAdvisor Location ID
df = pd.DataFrame(all_attractions).drop_duplicates(subset=["location_id"])
df.reset_index(drop=True, inplace=True)

# Details for each attraction:

attraction_rows = []

for i, item in df.iterrows():

    loc_id = item["location_id"]
    detail_url = DETAILS_URL.format(loc_id)

    params = {
        "key": API_KEY,
        "language": "en",
        "currency": "EUR"
    }

    detail_res = requests.get(detail_url, headers=headers, params=params)
    details = detail_res.json()
    address = details.get("address_obj", {}) or {}

    attraction_rows.append({
        "location_id": loc_id,
        "name": details.get("name"),
        "rating": details.get("rating"),
        "num_reviews": details.get("num_reviews"),
        "ranking": details.get("ranking"),
        "category": details.get("category", {}).get("name") if details.get("category") else None,
        "subcategory": ", ".join(
            [s["name"] for s in details.get("subcategory", [])]
        ) if details.get("subcategory") else None,
        "street1": address.get("street1"),
        "city": address.get("city"),
        "country": address.get("country"),
        "postalcode": address.get("postalcode"),
        "address_string": address.get("address_string"),
    })

    print(f"Retrieved: {details.get('name')} (rating: {details.get('rating')})")

    time.sleep(0.2)  # prevent rate limits

# Saving DataFrame:

df_attractions = pd.DataFrame(attraction_rows)

output_file = "lisbon_attractions_with_ratings.csv"
df_attractions.to_csv(output_file, index=False)

print(f"\nDONE! Saved {len(df_attractions)} attractions with ratings to {output_file}")


 - I got 70 attractions extracted.
-----------------------------------------------------------